In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

dtypes_raw = {
    'elapsed_time': np.int32,
    'event_name': 'category',
    'name': 'category',
    'level': np.uint8,
    'room_coor_x': np.float32,
    "index": np.int32,
    'room_coor_y': np.float32,
    'screen_coor_x': np.float32,
    'screen_coor_y': np.float32,
    'hover_duration': np.float32,
    'text': 'category',
    'fqid': 'category',
    'room_fqid': 'category',
    'text_fqid': 'category',
    'fullscreen': 'category',
    'hq': 'category',
    'music': 'category',
    'level_group': 'category'}

dtypes_pp = {
            'level': np.uint8,  
            'session_id': np.int64,
            'level_group': 'category',
            'event_name': np.uint8,
            'name': np.uint8,
            'fqid': np.uint8,
            'room_fqid': np.uint8,
            'text_fqid': np.uint8,
            'fullscreen': np.uint8,
            'hq': np.uint8,
            'music': np.uint8,
            'hover_duration_mean': np.float32,
            'difference_clicks_mean': np.float32,
            "distance_clicks_mean": np.float32,
            "screen_distance_clicks_mean": np.float32,            
            'elapsed_time_std': np.float32,
            'page_std': np.float32,
            'room_coor_x_std': np.float32,
            'room_coor_y_std': np.float32,
            'screen_coor_x_std': np.float32,
            'screen_coor_y_std': np.float32,
            'hover_duration_std': np.float32,
            'difference_clicks_std': np.float32,
            "distance_clicks_std": np.float32,
            "screen_distance_clicks_std": np.float32,
            'index_sum_of_actions': np.int32,
            'difference_clicks_max': np.float32,
            'elapsed_time_max': np.float32,
            'clicks_per_second': np.float32,
            "sum_distance_clicks_max": np.float32}

wd = os.getcwd()
print("Current working directory: ", wd)
if wd[-10:] == 'submission': wd = wd[:-11]
os.chdir(wd)
print("New working directory: ", os.getcwd())

Current working directory:  /Users/nzuchna/Desktop/Drive/2. Areas/University/Master-TUC/M2/4_Forschungsmodul/student-performance/submission
New working directory:  /Users/nzuchna/Desktop/Drive/2. Areas/University/Master-TUC/M2/4_Forschungsmodul/student-performance


# 1. Load your model: [your model name here] (in this case: steves optuna xgboost decision tree)

In [ ]:
import xgboost as xgb
import pickle

print("New working directory: ", os.getcwd())

# Load the model from file
with open('models/best_models_dict_optunahp.pkl', 'rb') as f:
    models = pickle.load(f)

In [2]:
!pip show xgboost

Name: xgboost
Version: 1.7.3
Summary: XGBoost Python Package
Home-page: https://github.com/dmlc/xgboost
Author: 
Author-email: 
License: Apache-2.0
Location: /Users/nzuchna/opt/anaconda3/envs/gameplay_kaggel_env/lib/python3.9/site-packages
Requires: numpy, scipy
Required-by: 


In [1]:
models['1']

NameError: name 'models' is not defined

# 2. Run the model
DON'T READ THIS: This jo_wilder-kind-of-timeseries-API simulates the real world scenario for the model you've trained. The iter_test gives you pairs of: 
1. X_test_raw: gamelog data of a single session_id from only one level_group 
2. y_test_label_preds: and secondly a dataframe (similiar to raw/train_labels.csv) in which you have to fill in your predictions for each question of that level_group.

**Additional WARNINGS**:
- submission.csv is NOT automatically reseted, even after a Kernel reboot the iter_test will continue to add sample submissions to the submission.csv. Therefor, before your final run, do a factory reset (Run > Factory reset).
- the iter_tester needs a kernel reboot after each run - or you will get the error: "Exception: You can only iterate over `iter_test()` once."

In [2]:
X_train = pd.read_csv('data/raw/train.csv')  #TODO: try with test.csv also
y_train = pd.read_csv('data/raw/train_labels.csv')

In [1]:
import sys
import io

# Import utils for feature engineering
from utils.data_preprocessing_steve import pp_pipeline_noah

limits = {'0-4':(1,4), '5-12':(4,14), '13-22':(14,19)}
questions = {'0-4':('_q1', '_q2', '_q3'), '5-12':('_q4', '_q5', '_q6', '_q7', '_q8', '_q9', '_q10', '_q11', '_q12', '_q13'), '13-22':('_q14', '_q15', '_q16', '_q17', '_q18')}
level_groups = ['0-4', '5-12', '13-22']

submission_df = pd.DataFrame()

# Simulate the time series API
for sess in tqdm(pd.unique(X_train.session_id.astype('str'))):  # Raw data from only ONE session_id and level_group in each loop
    # print('sess: '+sess)
    X_test_sess = X_train[X_train['session_id'] == int(sess)]
    y_test_sess = y_train[y_train['session_id'].str.contains(sess)]

    for grps in pd.unique(X_test_sess.level_group.astype('str')):
        #print('sess: '+sess+', grp: '+grps)

        X_test_raw = X_test_sess[X_test_sess['level_group'] == grps]
        group_endings = questions[grps]
        y_test_label_preds = y_test_sess[y_test_sess['session_id'].str.endswith(group_endings)]

        #print(X_test_raw.head())
        #print(y_test_label_preds.head())

        # Suppress the print's inside of steve's preprocessing functions...
        text_trap = io.StringIO()
        sys.stdout = text_trap

        # Pre-process the raw test data: the pipeline returns a dict with datasets split by level_group.
        X_test_dict = pp_pipeline_noah(data=X_test_raw, file_path=None, flatten=True, saveIntermediateFiles=False,
                         dtypes=dtypes_raw, output=True)

        # Restore stdout.
        sys.stdout = sys.__stdout__

        # Since we only have data from one level_group, we just take the first dataset out of the dict.
        grp = list(X_test_dict.keys())[0]
        X_test = X_test_dict[grp].drop(['session_id_1', 'level_group_1'], axis=1)

        # Here we adapt the dtypes of our processed data
        for column in X_test.columns:       # TODO: put this in a function and optimize it
            base_name = column.rsplit('_', 1)[0]  # get the base name by splitting on the last "_" character
            if base_name in dtypes_pp:
                column_number = column.rsplit('_', 1)[1]  # get the number from the index by splitting on the last "_" character
                new_column_name = f"{base_name}_{column_number}"  # construct the new column name
                column_dtype = dtypes_pp[base_name]
                try:
                    X_test[new_column_name] = X_test[column].astype(column_dtype)  # set the same data type for all columns with the same base name
                except:
                    X_test[new_column_name] = pd.Series(int(0)).astype(column_dtype)

        a,b = limits[grp]

        # We loop for each question - t
        for t in range(a,b):
            # Get the model for question t
            #model = models[f'{t}']     # TODO: load actual models
            # Make prediction
            y_preds = 1 #model.predict(X_test)
            # Write prediction to y_test_label_preds
            mask = y_test_label_preds.session_id.str.contains(f'q{t}')
            y_test_label_preds.loc[mask,'correct'] = y_preds    #[0]
            submission_df = pd.concat([submission_df, y_test_label_preds.loc[mask,'correct']])  # TODO: make submission_df actually mirror submission.csv

        # Add your predictions of this session_id to the submission before getting the next session_id data
        #print(submission_df.head(40))

submission_df.reset_index(inplace=True)

NameError: name 'pd' is not defined

# 3. Quick submission EDA
- Check if your submission file looks correct

In [5]:
#df = pd.read_csv('submission.csv')
print(submission_df.shape)
print(submission_df.head(20))
print(submission_df.correct.mean())

AttributeError: 'DataFrame' object has no attribute 'correct'

In [6]:
## the end result is a submission file containing all test session predictions
! head submission.csv

# 4. Final steps for submission
* In the top right corner choose "Save Version" with Version type "Save & Run All (Commit)". Note the correct Version for later.
* In the lower left corner you'll see the process running. When it's done, press on the "Open in viewer" option and check if the output is a submission.csv with 1.29 kB
* Then go to the competition page, choose "Submit Predictions", choose your file with the correct version and correct output file - submit and you're done.
* Your submitted notebook will now be re-runned with private test data by the competition host. The public socre will appear in your submission section.
* The scoring will take a while...